In [13]:
import json
import numpy as np
import pandas as pd
from pathlib import Path

SAMPLE_FP = Path("sample.jsonl")

def read_jsonl(fp: Path) -> pd.DataFrame:
    rows = []
    with fp.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                rows.append(json.loads(line))
    return pd.DataFrame(rows)

df = read_jsonl(SAMPLE_FP)

# --- Flatten dict columns ---
state = pd.json_normalize(df["state"]).add_prefix("st.")
pheno = pd.json_normalize(df["phenotype"]).add_prefix("ph.")
local = pd.json_normalize(df["local"]).add_prefix("loc.")
pos   = pd.json_normalize(df["pos"]).add_prefix("pos.")

g = pd.concat(
    [df.drop(columns=[c for c in ["state","phenotype","local","pos"] if c in df.columns]),
     state, pheno, local, pos],
    axis=1
)

# Ensure numerics
for c in ["t","age","agent_id","pop_n", "st.E","st.M","st.D","ph.A_mature","ph.E_repro_min","ph.M_repro_min","st.repro_cd_s"]:
    if c in g.columns:
        g[c] = pd.to_numeric(g[c], errors="coerce")

print("Flattened columns:", [c for c in g.columns if c.startswith(("st.","ph.","loc.","pos."))][:20], " ...")
display(g.head(3))

# --- Gates (match your new rule) ---
# energy gate: Et >= E_repro_min * M   (as you proposed)
Et   = g["st.E"]
M    = g["st.M"]
Amat = g["ph.A_mature"]
Emin = g["ph.E_repro_min"]
Mmin = g["ph.M_repro_min"]

g["ok_age"] = g["age"] >= Amat

# cooldown optional: if you don't log it, assume ok
if "st.repro_cd_s" in g.columns:
    g["ok_cd"] = g["st.repro_cd_s"].fillna(0.0) <= 0.0
else:
    g["ok_cd"] = True

g["E_gate_rhs"]    = Emin * M
g["energy_margin"] = Et - g["E_gate_rhs"]
g["ok_E"]          = g["energy_margin"] >= 0.0

g["mass_margin"] = M - Mmin
g["ok_M"]        = g["mass_margin"] >= 0.0

g["ok_ready"] = g["ok_age"] & g["ok_cd"] & g["ok_E"] & g["ok_M"]

def share(s): 
    s = s.dropna()
    return float(s.mean()) if len(s) else float("nan")

print("\nShares:")
print(pd.Series({
    "ok_age": share(g["ok_age"]),
    "ok_cd": share(g["ok_cd"]),
    "ok_E": share(g["ok_E"]),
    "ok_M": share(g["ok_M"]),
    "ok_ready": share(g["ok_ready"]),
}).round(4))

print("\nKey describes:")
display(g[["st.E","st.M","ph.A_mature","ph.E_repro_min","ph.M_repro_min","energy_margin","mass_margin"]].describe().round(6))

# Which gate fails first?
def first_fail(r):
    if not r["ok_age"]: return "age"
    if not r["ok_cd"]:  return "cooldown"
    if not r["ok_E"]:   return "energy"
    if not r["ok_M"]:   return "mass"
    return "ready"

g["fail_reason"] = g.apply(first_fail, axis=1)
display(g["fail_reason"].value_counts().to_frame("count"))

# Per-agent readiness rate (useful)
per_agent = g.groupby("agent_id").agg(
    n=("ok_ready","size"),
    share_ready=("ok_ready","mean"),
    share_okE=("ok_E","mean"),
    share_okM=("ok_M","mean"),
    M_med=("st.M","median"),
    Mreq_med=("ph.M_repro_min","median"),
    Et_med=("st.E","median"),
    Emin_med=("ph.E_repro_min","median"),
    Amat=("ph.A_mature","median"),
).sort_values(["share_ready","share_okM","share_okE"], ascending=False)
display(per_agent.round(4))

Flattened columns: ['st.E', 'st.E_fast', 'st.E_slow', 'st.M', 'st.Fg', 'st.D', 'st.hunger', 'st.speed', 'st.repro_cd_s', 'ph.A_mature', 'ph.repro_rate', 'ph.E_repro_min', 'ph.repro_cost', 'ph.M_repro_min', 'ph.E_rep_min', 'ph.metabolism_scale', 'ph.susceptibility', 'ph.stress_per_drain', 'ph.repair_capacity', 'ph.frailty_gain']  ...


,event,t,agent_id,age,birth_t,traits,pop_n,st.E,st.E_fast,st.E_slow,...,ph.sociability,ph.mobility,ph.cold_aversion,ph.sense_strength,loc.B0,loc.F0,loc.C0,pos.x,pos.y,pos.heading
0,sample,0.0,3,0.02,0.0,"[-0.6437026262283325, 0.751379132270813, 0.353...",12,0.192495,0.150664,0.255240,...,0.478837,0.708768,0.453721,0.727475,1.000000,0.012893,0.023042,35.873061,31.556621,-2.651120
1,sample,1.0,6,1.02,0.0,"[0.11625058203935623, 0.8801883459091187, -0.5...",12,0.133437,0.133061,0.134001,...,0.341044,0.286676,0.455312,0.392084,0.441086,0.046055,0.026810,30.973413,24.094018,2.877159
2,sample,2.0,4,2.02,0.0,"[0.4509298801422119, 0.7387500405311584, -0.07...",12,0.145119,0.167369,0.111744,...,0.574710,0.620196,0.549161,0.508236,0.806585,0.000000,0.023334,30.980999,33.627623,-0.898838



Shares:
ok_age      0.9130
ok_cd       0.9306
ok_E        0.8575
ok_M        0.5282
ok_ready    0.4755
dtype: float64

Key describes:


,st.E,st.M,ph.A_mature,ph.E_repro_min,ph.M_repro_min,energy_margin,mass_margin
count,1081.000000,1081.000000,1081.000000,1081.000000,1081.000000,1081.000000,1081.000000
mean,0.164963,0.319221,19.493584,0.215662,0.319607,0.096318,-0.000386
std,0.070539,0.063798,2.413647,0.020194,0.016455,0.065595,0.066627
min,0.000000,0.106862,14.916072,0.154229,0.257027,-0.100210,-0.228738
25%,0.150433,0.286539,18.635883,0.222273,0.315520,0.093630,-0.032096
50%,0.186468,0.321006,18.738459,0.223243,0.316303,0.118863,0.003457
75%,0.211707,0.359829,18.738459,0.223243,0.316505,0.134250,0.041937
max,0.275506,0.469483,29.879849,0.232420,0.374179,0.189151,0.184634


,count
fail_reason,
ready,514
mass,285
energy,154
age,94
cooldown,34


,n,share_ready,share_okE,share_okM,M_med,Mreq_med,Et_med,Emin_med,Amat
agent_id,,,,,,,,,
41,3,1.0,1.0,1.0,0.3616,0.3155,0.2167,0.2232,18.7385
46,1,1.0,1.0,1.0,0.3221,0.3165,0.1088,0.2232,18.7385
80,1,1.0,1.0,1.0,0.3495,0.3165,0.2094,0.2223,18.7385
93,2,1.0,1.0,1.0,0.3636,0.3165,0.2179,0.2227,18.7385
130,2,1.0,1.0,1.0,0.3429,0.3165,0.2054,0.2223,18.7385
...,...,...,...,...,...,...,...,...,...
129,1,0.0,0.0,0.0,0.2749,0.3165,0.0000,0.2223,18.7385
140,1,0.0,0.0,0.0,0.2770,0.3191,0.0000,0.2232,18.6359
143,1,0.0,0.0,0.0,0.2964,0.3163,0.0311,0.2232,18.7354


In [14]:
dt = 0.02  # din AP.dt
ready = g[g["ok_ready"]]

print(ready["ph.repro_rate"].describe())
p_step = 1.0 - np.exp(-ready["ph.repro_rate"] * dt)
print(p_step.describe())

# Grov "förväntad antal triggers" i samplet (inte exakt, men ska INTE vara ~0)
print("E[triggers] approx over rows:", float(p_step.sum()))

count    514.000000
mean       0.400564
std        0.013630
min        0.281781
25%        0.403521
50%        0.403521
75%        0.403521
max        0.412058
Name: ph.repro_rate, dtype: float64
count    514.000000
mean       0.007979
std        0.000271
min        0.005620
25%        0.008038
50%        0.008038
75%        0.008038
max        0.008207
Name: ph.repro_rate, dtype: float64
E[triggers] approx over rows: 4.101330215839951


In [15]:
import pandas as pd

life = pd.read_json("life.jsonl", lines=True)

life["event"].value_counts()


event
birth    154
death    137
Name: count, dtype: int64

In [16]:
life[life["event"] == "birth"][["t", "agent_id"]].sort_values("t")

,t,agent_id
0,0.00,1
11,0.00,12
10,0.00,11
9,0.00,10
7,0.00,8
...,...,...
275,1938.54,150
280,1952.44,151
281,1954.14,152
288,1977.14,153
